In [1]:
df = spark.read.option("multiline", "true").json("Files/bing-api-data.json")
# df now is a Spark DataFrame containing JSON data from "Files/bing-api-data.json".
display(df)

StatementMeta(, ed6c4c00-8508-431d-9cc4-212aac9662e5, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fdb74a11-f7e9-4c16-b7b7-9cc089461c35)

In [2]:
df = df.select("value")
display(df)

StatementMeta(, ed6c4c00-8508-431d-9cc4-212aac9662e5, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9fdd61db-c2e0-44a7-b715-fe088f2952c7)

In [3]:
from pyspark.sql.functions import explode
df_exploded = df.select(explode(df['value']).alias('json_object'))

StatementMeta(, ed6c4c00-8508-431d-9cc4-212aac9662e5, 5, Finished, Available, Finished)

In [4]:
display(df_exploded)

StatementMeta(, ed6c4c00-8508-431d-9cc4-212aac9662e5, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f082ab22-bb62-4623-a645-82ee8d13efc5)

In [5]:
json_list = df_exploded.toJSON().collect()

StatementMeta(, ed6c4c00-8508-431d-9cc4-212aac9662e5, 7, Finished, Available, Finished)

In [6]:
print(json_list)

StatementMeta(, ed6c4c00-8508-431d-9cc4-212aac9662e5, 8, Finished, Available, Finished)

['{"json_object":{"category":"Entertainment","datePublished":"2025-01-18T22:41:06.0000000Z","description":"Kesha shared an Instagram post joking that her fans will still have a \'TiK ToK\' to listen to after the Supreme Court announced its decision to allow the ban on the social media platform TikTok to go into effect on Jan.","name":"Kesha Reminds Fans Her Hit ‘TiK ToK’ \'Is Forever\' amid Ban of Social Media Platform TikTok","provider":[{"_type":"Organization","image":{"thumbnail":{"contentUrl":"https://www.bing.com/th?id=ODF.kridaNmIIz5AdgcUGG5eBQ&pid=news"}},"name":"People on MSN.com"}],"url":"https://www.msn.com/en-us/entertainment/celebrities/kesha-reminds-fans-her-hit-tik-tok-is-forever-amid-ban-of-social-media-platform-tiktok/ar-AA1xraQu"}}', '{"json_object":{"datePublished":"2025-01-19T10:58:00.0000000Z","description":"A new law has effectively forced Chinese tech company ByteDance to take TikTok and several other apps offline in various ways. And, when storefronts run by Appl

In [7]:
print(json_list[0])

StatementMeta(, ed6c4c00-8508-431d-9cc4-212aac9662e5, 9, Finished, Available, Finished)

{"json_object":{"category":"Entertainment","datePublished":"2025-01-18T22:41:06.0000000Z","description":"Kesha shared an Instagram post joking that her fans will still have a 'TiK ToK' to listen to after the Supreme Court announced its decision to allow the ban on the social media platform TikTok to go into effect on Jan.","name":"Kesha Reminds Fans Her Hit ‘TiK ToK’ 'Is Forever' amid Ban of Social Media Platform TikTok","provider":[{"_type":"Organization","image":{"thumbnail":{"contentUrl":"https://www.bing.com/th?id=ODF.kridaNmIIz5AdgcUGG5eBQ&pid=news"}},"name":"People on MSN.com"}],"url":"https://www.msn.com/en-us/entertainment/celebrities/kesha-reminds-fans-her-hit-tik-tok-is-forever-amid-ban-of-social-media-platform-tiktok/ar-AA1xraQu"}}


In [8]:
print(json_list[46])

StatementMeta(, ed6c4c00-8508-431d-9cc4-212aac9662e5, 10, Finished, Available, Finished)

{"json_object":{"category":"ScienceAndTechnology","datePublished":"2025-01-18T22:36:00.0000000Z","description":"TikTok is set to be banned tomorrow. Here's what time the ban could start in the U.S. and what the app could look like for users trying to access it on Sunday.","name":"What Time Will The TikTok Ban Start In The U.S.? Here’s What Could Happen","provider":[{"_type":"Organization","image":{"thumbnail":{"contentUrl":"https://www.bing.com/th?id=ODF.U6nYa4Huf-RFEhjg-V4hNg&pid=news"}},"name":"Forbes"}],"url":"https://www.forbes.com/sites/monicamercuri/2025/01/18/what-time-will-the-tiktok-ban-start-in-the-us-heres-what-could-happen/"}}


In [9]:
import json
new_json = json.loads(json_list[0])

StatementMeta(, ed6c4c00-8508-431d-9cc4-212aac9662e5, 11, Finished, Available, Finished)

### Processing the JSON property to List

In [10]:
title = []
description = []
category = []
url =[]
image = []
provider = []
datePublished =[]


# Process each JSON object in the list
for json_str in json_list:
    try:
        # Parse the JSON string into a dictionary
        article = json.loads(json_str)

        if article["json_object"].get("category") and article["json_object"].get("image", {}).get("thumbnail", {}).get("contentUrl"):
        
            #Extract information from the dictionary
            title.append(article["json_object"]["name"])
            description.append(article["json_object"]["description"])
            category.append(article["json_object"]["category"])
            url.append(article["json_object"]["url"])
            image.append(article["json_object"]["image"]["thumbnail"]["contentUrl"])
            provider.append(article["json_object"]["provider"][0]['name'])
            datePublished.append(article["json_object"]["datePublished"])

    except Exception as e:
        print(f"Error processing JSON object: {e}")

StatementMeta(, ed6c4c00-8508-431d-9cc4-212aac9662e5, 12, Finished, Available, Finished)

#### Converting the List to a Dataframe

In [11]:
from pyspark.sql.types import StructType, StructField, StringType


# Combine the lists
data = list(zip(title,description,category,url,image,provider,datePublished))

# Define schema
schema = StructType([
    StructField("title", StringType(), True),
    StructField("description", StringType(), True),
    StructField("category", StringType(), True),
    StructField("url", StringType(), True),
    StructField("image", StringType(), True),
    StructField("provider", StringType(), True),
    StructField("datePublished", StringType(), True)
])

# Create DataFrame
df_cleaned = spark.createDataFrame(data, schema=schema)

StatementMeta(, ed6c4c00-8508-431d-9cc4-212aac9662e5, 13, Finished, Available, Finished)

In [12]:
display(df_cleaned)

StatementMeta(, ed6c4c00-8508-431d-9cc4-212aac9662e5, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 66b074b0-2bf5-4371-8e15-7e7d10b0d455)

In [13]:
from pyspark.sql.functions import to_date, date_format

df_cleaned_final = df_cleaned.withColumn("datePublished", date_format(to_date("datePublished"), "dd-MMM-yyyy"))

StatementMeta(, ed6c4c00-8508-431d-9cc4-212aac9662e5, 15, Finished, Available, Finished)

In [14]:
display(df_cleaned_final)

StatementMeta(, ed6c4c00-8508-431d-9cc4-212aac9662e5, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d3a59ffa-b6cd-42ca-9cda-1f024e3af538)

### Writing the Final Dataframe to the Lakehouse DB in a Delta format

In [15]:
from pyspark.sql.utils import AnalysisException

try:

    table_name = 'bing_lake_db.tbl_latest_news'

    df_cleaned_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table Already Exists")

    df_cleaned_final.createOrReplaceTempView("vw_df_cleaned_final")

    spark.sql(f"""  MERGE INTO {table_name} target_table
                    USING vw_df_cleaned_final source_view

                    ON source_view.url = target_table.url
                    
                    WHEN MATCHED AND 
                    source_view.title <> target_table.title OR
                    source_view.description <> target_table.description OR
                    source_view.category <> target_table.category OR
                    source_view.image <> target_table.image OR
                    source_view.provider <> target_table.provider OR
                    source_view.datePublished <> target_table.datePublished   

                    THEN UPDATE SET *

                    WHEN NOT MATCHED THEN INSERT *

                """)

StatementMeta(, ed6c4c00-8508-431d-9cc4-212aac9662e5, 17, Finished, Available, Finished)

Table Already Exists


In [16]:
%%sql

SELECT count(*) from bing_lake_db.tbl_latest_news

StatementMeta(, ed6c4c00-8508-431d-9cc4-212aac9662e5, 18, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>